# Эндоморфизмы эллиптических кривых и проблема дискретного логарифма. Решение задачи дискретного логарифмирования в частном случае

## Некоторые необходимые для создания классов Curve и Point функции 

In [1]:
from random import randint
from sympy import randprime 
import time

def st(n):
    s = 0
    t = n
    while t % 2 == 0:
        s += 1
        t = t // 2
    return s, t

def Jacobi(a,n):
    if n < 0 or not n % 2:
        raise ValueError("n should be an odd positive integer") 
    j = 1
    if n == 1:
        return j
    if a<0:
        a = -a
        if n%4 == 3:
            j = -j
    while n>1:
        if a == 0:
            return 0
        s,t = st(a)
        if (s%2 == 1) & (n%8 in [3, 5]):
            j = -j
        if 3 == n%4 == t%4:
            j = -j
        a = n%t
        n = t
    return j

def Shanks(a, p):
    if a % p == 0:
        return 0
    if not Jacobi(a, p) == 1: 
        return "no solution"
    s,t = st(p-1)             
    n = randint(2,p-2) 
    while Jacobi(n,p) == 1:   
        n = randint(2,p-2)
    b = pow(n,t,p)            
    r = pow(a,(t+1)//2,p)     
    d = 0
    f = pow(a,t,p)            
    b2 = b
    for i in range(1,s):      
        b2 = b2*b2 % p        
        if not pow(f,2**(s-1-i), p) == 1:  
            d += 2**i                   
            f = f*b2 % p                 
    return r*pow(b,d//2,p) % p          

In [2]:
def D(a,b):
    return 27*b**2 + 4*a**3

# Реализация классов Point (infPoint) и Curve для работы с точками эллиптической кривой

In [3]:
class Point:
    def __init__(self,x,y, curve):
        self.curve = curve
        x %= self.curve.p
        y %= self.curve.p
        if not self.curve.isPoint(x,y):
            print("Ошибка! Точка не лежит на кривой!")
        else:
            self.x = x 
            self.y = y 
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.x == other.x) and (self.y == other.y) and (self.curve == other.curve)
        else:
            return False
    
    def __repr__(self):
        return f"({self.x}, {self.y})"
    
    def __neg__(self):
        return Point(self.x, -self.y, self.curve)
    
    def doubling(self):
        y1 = self.y
        x1 = self.x
        if y1 == 0:
            return infPoint(self.curve)
        else:
            k = (3*(x1*x1 % self.curve.p) + self.curve.a)*pow((2*y1),-1, self.curve.p) % self.curve.p  
            x3 = k*k % self.curve.p - 2*x1            
            y3 = k*(x1-x3) - y1        
            return Point(x3,y3, self.curve)
        
    def __add__(self, P):
        if isinstance(P, infPoint):
            return self
        y1 = self.y
        x1 = self.x
        y2 = P.y
        x2 = P.x
        if x1 == x2:
            if y1 == y2 == 0:
                return infPoint(self.curve)
            elif y1 != y2:
                return infPoint(self.curve)
            else:
                k = (3*(x1*x1 % self.curve.p) + self.curve.a)*pow((2*y1),-1, self.curve.p) % self.curve.p  
                x3 = k*k % self.curve.p - 2*x1            
                y3 = k*(x1-x3) - y1        
                return Point(x3,y3, self.curve)
        k = (y2-y1)*pow((x2 - x1), -1, self.curve.p) % self.curve.p       
        x3 = k*k % self.curve.p - x1- x2                        
        y3 = k*(x1-x3) - y1                     
        return Point(x3,y3, self.curve)
        
    def __sub__(self,P):
        return self + (-P)
    
    def __mul__(self, m):
        if isinstance(m, int):
            if m < 0:
                m = -m
                self = -self
            m = bin(m)[2:]
            result = infPoint(self.curve)
            Q = self
            for i in m[::-1]:
                if i == "1":
                    result = result + Q
                Q += Q
            return result
        else:
            return f"{m} должно быть целым числом"
        
    def __rmul__(self,m):
        return self*m

In [4]:
class infPoint():
    def __init__(self, curve):
        self.curve = curve
    def __neg__(self):
        return self  
    def __add__(self, P):
        return P
    def __sub__(self,P):
        return self + (-P)
    def __repr__(self):
        return "O"
    def __mul__(self, m):
        return self
    def __rmul__(self, m):
        return self
    def toProjective(self):
        return ProjectivePoint(0,1,0,self.curve.toProjective())
    def toJacobian(self):
        return JacobianPoint(1,1,0,self.curve.toJacobian())
    def toChudnovskyJacobian(self):
        return ChudnovskyJacobianPoint(1,1,0,0,0,self.curve.toChudnovskyJacobian())

In [5]:
class Curve:
    def __init__(self,a,b,p):
        if D(a,b) % p: 
            self.a = a % p
            self.b = b % p
            self.p = p
        else:
            print("Ошибка! Дискриминант кривой нулевой!")
            
    def __eq__(self, other):
        if isinstance(other, self.__class__):
            return (self.a == other.a) and (self.b == other.b) and (self.p == other.p)
        else:
            return False
    
    def findPointsShanks(self):
        points = []
        for x in range(self.p):
            a = (pow(x,3,self.p) + self.a*x + self.b) % self.p
            y = Shanks(a, self.p)
            if isinstance(y, int):
                points.append(Point(x,y,self))
                if y != ((self.p - y) % self.p):
                    points.append(Point(x,-y, self))
        points.append(infPoint(self))
        return points
    
    def findRandomPoint(self):
        while True:
            x = randint(0, self.p - 1)
            a = (pow(x,3,self.p) + self.a*x + self.b) % self.p
            y = Shanks(a, self.p)
            if isinstance(y, int):
                return Point(x,y, self)
    
    def isPoint(self,x,y):
        return not ((pow(y,2,self.p) - (pow(x,3,self.p) + self.a*x + self.b)) % self.p)
        
    def toProjective(self):
        return ProjectiveCurve(self.a,self.b,self.p)
    
    def toJacobian(self):
        return JacobianCurve(self.a,self.b,self.p)
    
    def toChudnovskyJacobian(self):
        return ChudnovskyJacobianCurve(self.a,self.b,self.p)
    
    def curveOrder(self):
        return len(self.findPointsShanks())

# Реализация алгоритмов для вычисления дискретного логарифма эллиптической кривой

Исходный код функции `nthroot_mod` из `sympy.ntheory.residue_ntheory`

In [6]:
from sympy.core.compatibility import as_int
from sympy.core.numbers import igcd, igcdex, mod_inverse
from sympy.ntheory.residue_ntheory import is_nthpow_residue
from sympy.ntheory.residue_ntheory import primitive_root
from sympy.ntheory.residue_ntheory import _nthroot_mod1
def nthroot_mod(a, n, p, all_roots = False):
    """
    Find the solutions to ``x**n = a mod p``

    Parameters
    ==========

    a : integer
    n : positive integer
    p : positive integer
    all_roots : if False returns the smallest root, else the list of roots

    Examples
    ========

    >>> from sympy.ntheory.residue_ntheory import nthroot_mod
    >>> nthroot_mod(11, 4, 19)
    8
    >>> nthroot_mod(11, 4, 19, True)
    [8, 11]
    >>> nthroot_mod(68, 3, 109)
    23
    """
    from sympy.core.numbers import igcdex
    a, n, p = as_int(a), as_int(n), as_int(p)
    if n == 2:
        return sqrt_mod(a, p, all_roots)
    # see Hackman "Elementary Number Theory" (2009), page 76
    if not is_nthpow_residue(a, n, p):
        return None
    if primitive_root(p) is None:
        raise NotImplementedError("Not Implemented for m without primitive root")

    if (p - 1) % n == 0:
        return _nthroot_mod1(a, n, p, all_roots)
    # The roots of ``x**n - a = 0 (mod p)`` are roots of
    # ``gcd(x**n - a, x**(p - 1) - 1) = 0 (mod p)``
    pa = n
    pb = p - 1
    b = 1
    if pa < pb:
        a, pa, b, pb = b, pb, a, pa
    while pb:
        # x**pa - a = 0; x**pb - b = 0
        # x**pa - a = x**(q*pb + r) - a = (x**pb)**q * x**r - a =
        #             b**q * x**r - a; x**r - c = 0; c = b**-q * a mod p
        q, r = divmod(pa, pb)
        c = pow(b, q, p)
        c = igcdex(c, p)[0]
        c = (c * a) % p
        pa, pb = pb, r
        a, b = b, c
    if pa == 1:
        if all_roots:
            res = [a]
        else:
            res = a
    elif pa == 2:
        return sqrt_mod(a, p , all_roots)
    else:
        res = _nthroot_mod1(a, pa, p, all_roots)
    return res


Зададим функцию для использования эндоморфизма $\phi: (x, y) \mapsto(\beta x, y)$ и $\mathcal{O} \mapsto \mathcal{O}$

In [7]:
def end(p, beta):
    if isinstance(p, infPoint):
        return infPoint(p.curve)
    return Point(p.x*beta, p.y, p.curve)

1. Решение уравнения $xG = P_1$ по известному решению уравнения $xG = P_2$, где $P_1$ и $P_2$ имеют одинаковую координату $y$.

In [8]:
def dLogSolver(p, n, P1, P2, t, b = None, l = None):
    if not b:
        b = nthroot_mod(1, 3, p, True)[1]
    if not l:
        l = nthroot_mod(1, 3, n, True)[1]
    if end(P1, b) == P2:
        x = t*pow(l,-1, n) % n
    else:
        x = t*pow(l**2, -1, n) % n
    return x

2. Решение уравнения $xG = P_1$ по известному решению уравнения $xG = P_2 - P_1$, где $P_1$ и $P_2$ имеют одинаковую координату $y$.

In [9]:
def dLogSolverDif(p, n, P1, dif, t, b = None, l = None):
    P2 = dif + P1
    if not b:
        b = nthroot_mod(1, 3, p, True)[1]
    if not l:
        l = nthroot_mod(1, 3, n, True)[1]
    if end(P1, b) == P2:
        x = t*pow(l - 1,-1, n) % n
    else:
        x = t*pow(l**2 - 1, -1, n) % n
    return x

## Тестирование алгоритмов на кривой небольшого порядка

In [10]:
p = 43
p % 3

1

In [11]:
c = Curve(0,7,p)
c.findPointsShanks()

[(2, 31),
 (2, 12),
 (7, 36),
 (7, 7),
 (12, 31),
 (12, 12),
 (13, 21),
 (13, 22),
 (20, 40),
 (20, 3),
 (21, 25),
 (21, 18),
 (25, 25),
 (25, 18),
 (29, 31),
 (29, 12),
 (32, 40),
 (32, 3),
 (34, 40),
 (34, 3),
 (35, 21),
 (35, 22),
 (37, 36),
 (37, 7),
 (38, 21),
 (38, 22),
 (40, 25),
 (40, 18),
 (42, 36),
 (42, 7),
 O]

In [12]:
n = c.curveOrder()
n % 3

1

Рассмотрим первую задачу.

Выберем следующие исходные данные:
+ $G = (12, 31)$
+ $P_1 = (32, 40)$
+ $P_2 = 23G = (20, 40)$

In [13]:
G = Point(12, 31, c)
P1 = Point(32, 40, c)
P2 = Point(20, 40, c)
t = 23

In [14]:
sol1 = dLogSolver(p, n, P1, P2, t)
sol1 

17

In [15]:
print(G*sol1, P1)
G*sol1 == P1 # получили верный ответ

(32, 40) (32, 40)


True

Для решения второй задачи выберем новые исходные данные:
+ $G = (42, 36)$
+ $P_1 = (25,18)$
+ $P_2 = (21,18)$
+ $(P2 - P1) = 22G = (35, 22)$

In [16]:
G = Point(42,36,c)
P1 = Point(25,18,c)
P2 = Point(21,18,c)
t = 22

In [17]:
sol2 = dLogSolverDif(p, n, P1, P2 - P1, t)

In [18]:
print(sol2*G, P1)
G*sol2 == P1 # получили верный ответ

(25, 18) (25, 18)


True

## Тестирование алгоритмов на кривой SECP256k1

In [19]:
p = 115792089237316195423570985008687907853269984665640564039457584007908834671663
SECP256k1 = Curve(0, 7, p)
n = 115792089237316195423570985008687907852837564279074904382605163141518161494337
b = 55594575648329892869085402983802832744385952214688224221778511981742606582254
l = 37718080363155996902926221483475020450927657555482586988616620542887997980018

In [20]:
p%3, n%3

(1, 1)

Зададим следующие параметры:
+ $G = (43972058008956600510070150284766821022977458755289054035007856031232840643000, 51855161223576362722185940022860364125581297957735844065328061563388767261889)$
+ $P_1 = (106425298635331082471917436389928683282463065844388670776519822248290723610944, 34945961817953480676324849611701869571657592225822792221501648520056584636503)$
+ $P_2 = 876349152G =(99611287690666069782340804267622554666571445305230259955467091930852038341810, 34945961817953480676324849611701869571657592225822792221501648520056584636503)$

In [21]:
G = Point(43972058008956600510070150284766821022977458755289054035007856031232840643000, 51855161223576362722185940022860364125581297957735844065328061563388767261889, SECP256k1)
P1 = Point(106425298635331082471917436389928683282463065844388670776519822248290723610944, 34945961817953480676324849611701869571657592225822792221501648520056584636503, SECP256k1)
P2 = Point(99611287690666069782340804267622554666571445305230259955467091930852038341810, 34945961817953480676324849611701869571657592225822792221501648520056584636503, SECP256k1)
t = 876349152

In [22]:
sol3 = dLogSolver(p, n, P1, P2, t, b, l)
sol3

58954822210603468252214086162047578713591482914170349323264093426055053402803

In [23]:
sol3*G == P1 #получили верный ответ

True

Для второй задачи выберем другие данные:
+ $G = (39461403700986745438887632700736590506022318167932797587556248728463688478273,5167084641483602694706148308472026672006224131072422397043928705280825404911)$
+ $P_1 = (29504267309223397090056414194879693394903635186795986539925375262018400605330, 2700387467372297884102013513072671864733541412877449417142217900456736081571)$
+ $P_2 = 35416965966288193029253887080699235993332046944537972994674884479846800530688, 2700387467372297884102013513072671864733541412877449417142217900456736081571)$
+ $P_2 - P_1 = 2G = (22652517870328640072629002346542970354171872300069809619334405179449632131455, 37389770910085060872490402131497203789856670372833309420913746515394634041417)$

In [24]:
G = Point(39461403700986745438887632700736590506022318167932797587556248728463688478273,5167084641483602694706148308472026672006224131072422397043928705280825404911, SECP256k1)
P1 = Point(29504267309223397090056414194879693394903635186795986539925375262018400605330, 2700387467372297884102013513072671864733541412877449417142217900456736081571, SECP256k1)
P2 = Point(35416965966288193029253887080699235993332046944537972994674884479846800530688, 2700387467372297884102013513072671864733541412877449417142217900456736081571, SECP256k1)
t = 2

In [25]:
sol4 = dLogSolverDif(p, n, P1, P2-P1, t, b, l)
sol4

52049339249440132347096509016808591601273271149061544929325695065753442342878

In [26]:
G*sol4 == P1 # получили верный ответ

True